# Term Project : TSP

Group Piton

Ali Eren Pirli, Yunus Emre Aydın, Tuan Naci Çilingir, Serva Pınarbaşı

In [1]:
#We import the necessary modules

import urllib.request as ur
import pandas as pd
import math 
from gurobipy import * #Solver module for solving a mathematical model (for installing please check http://www.gurobi.com/documentation/8.0/quickstart_mac/installing_the_anaconda_py.html)
import folium #Data visualisation package(for installing please check https://anaconda.org/conda-forge/folium)
import matplotlib.pyplot as plt

In [2]:
#Taking input from the user for selecting which dataset to use in the model
print("Available datasets:")
print("1) 535 Airports around the globe (Padberg/Rinaldi)")
print("2) Europe-Subproblem of 666-city TSP (Groetschel)")
while True:
    try:
        choice= int(input("Please select one of the above data sets by entering 1 or 2: "))
        if choice == 1 or choice == 2:
            print("You choose dataset ", choice)
            break
        else:
            print("Invalid number!")
    except:
        print("Invalid character!")        

Available datasets:
1) 535 Airports around the globe (Padberg/Rinaldi)
2) Europe-Subproblem of 666-city TSP (Groetschel)
Please select one of the above data sets by entering 1 or 2: 1
You choose dataset  1


In [3]:
#There are different types of data files in the TSP Library.
#Depending on the input we will be using "gr202" case with 202 cities or "ali535" case with 535 airports around the world.
#We will import the data from tsp library using urlopen.

url_1= "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/ali535.tsp"
url_2= "http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/gr202.tsp"

if choice == 1:
    url= url_1
else:
    url= url_2

#Reading the file and saving each line into a list
try:
    with ur.urlopen(url) as f:
        data= f.readlines()
        
    #Transforming each line into "utf-8" format and removing the \n characters at the end of each file
    for i in range(len(data)):
        data[i]= data[i].decode("utf-8")
        data[i]= data[i].strip()
except:
    print("Error when importing the file! Please check the URL.")


Error when importing the file! Please check the URL.


In [4]:
#Printing the data for visualising

for  line in data:
    print(line)

NameError: name 'data' is not defined

# Because of the problem size and time consumption, we limit the size to maximum 150 nodes in "ali 535" case. Otherwise program crashes when trying to solve it in full size.

In [5]:
#First of all we just need coordinates, so we need to remove first couple of lines
#Depending on users choice we rearrange the data(if the choice is 1 then we need to limit the number of nodes to 150) 

if choice== 1:
    for i in range(len(data)):
        if data[i][0] == "1": #To understand where the data for the nodes starts
            data= data[i:i+150] #Limiting the number of nodes
            break
else:
    for i in range(len(data)):
        if data[i][0] == "1": #To understand where the data for the nodes starts
            data= data[i:len(data)-1] #Starting with firt node and removing the last line which contains "EOF"
            break
#Printing again to check
for line in data:
    print(line)

NameError: name 'data' is not defined

In [6]:
#Now we need modify the data and save it into a DataFrame object

latitude = []
longitude = []

#Saving the coordinate values as floats to different lists
for i in range(len(data)):
    line = data[i].strip()
    values = line.split()
    latitude.append(float(values[1]))
    longitude.append(float(values[2]))
    
#Saving the coordinate values as tuples into a list and making a DataFrame object   
coords= zip(latitude,longitude)
new_data= []
for node in coords:
    new_data.append(node)

df= pd.DataFrame(new_data, columns= ["Latitude", "Longitude"])

NameError: name 'data' is not defined

In [7]:
#Printing the DataFrame object
print(df)

NameError: name 'df' is not defined

# For converting coordinate input to longitude and latitude in radian, we use the following method:

PI = 3.141592

deg = (int) x[i] 

min = x[i]- deg 

rad = PI * (deg + 5.0 * min/ 3.0) / 180.0 

Reference: https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/TSPFAQ.html

In [8]:
#To implement the above equations, we define a function.
#We would use this function in order to calculate the distances between cities
def rad(x): 
    deg= math.modf(x)[1] #With math.modf() funtion we can get the integer and the fractional parts of a number
    m= math.modf(x)[0]
    return 3.141592*(deg + 5.0 * m/ 3.0) / 180.0

# To calculate the geographical distance between two points, we use the following method
q1 = cos( longitude[i] - longitude[j] )

q2 = cos( latitude[i] - latitude[j] ) 

q3 = cos( latitude[i] + latitude[j] ) 

dij = (int) ( 6378.388 * acos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)

Reference: https://wwwproxy.iwr.uni-heidelberg.de/groups/comopt/software/TSPLIB95/TSPFAQ.html


In [9]:
#To implement the aabove equations, we define a function
#To turn latitude and longitude values into radian, we use our pre-defined rad() function
def distance(df, i, j):
    q1= math.cos(rad(df["Longitude"][i]) - rad(df["Longitude"][j]))
    q2= math.cos(rad(df["Latitude"][i]) - rad(df["Latitude"][j]))
    q3= math.cos(rad(df["Latitude"][i]) + rad(df["Latitude"][j]))
    return math.modf(6378.388 * math.acos( 0.5*((1.0+q1)*q2 - (1.0-q1)*q3) ) + 1.0)[1]

In [10]:
#We construct the cost matrix that we will use in our model
c= {}
n= len(df)
for i in range(n):
    for j in range(n):
        c[(i,j)]= distance(df,i,j) #We use tuples as keys. Since they are immutable, we can use them as keys.

NameError: name 'df' is not defined

In [11]:
#Printing the cost matrix
print(c)

{}


# We have our data ready to use for our model

# Constructing the model 

In [12]:
#In the solution, we just want to have one tour that passes all nodes just once.
#We use pre-defined objects and functions in gurobi solver in order to define a function that eliminates subtours.
#http://examples.gurobi.com/traveling-salesman-problem/#demo

def subtourelim(model, where):
    if where == GRB.callback.MIPSOL:
        selected= []
        # To Make a list of edges selected in the solution
        for i in range(n):
            sol = model.cbGetSolution([model._x[i,j] for j in range(n)])
            selected += [(i,j) for j in range(n) if sol[j] > 0.5]
        # To find the shortest cycle in the selected edge list
        tour = subtour(selected)
        if len(tour) < n:
            # add a DFJ constraint (a constraint that eliminates a spesific subtour)
            expr = 0
            for i in range(len(tour)):
                for j in range(i+1, len(tour)):
                    expr += model._x[tour[i], tour[j]]
            model.cbLazy(expr <= len(tour)-1)

In [13]:
# Given a list of edges, finds the shortest subtour

def subtour(edges):
    visited = [False]*n
    cycles = []
    lengths = []
    selected = [[] for i in range(n)]
    for x,y in edges:
        selected[x].append(y)
    while True:
        current = visited.index(False)
        thiscycle = [current]
        while True:
            visited[current] = True
            neighbors = [x for x in selected[current] if not visited[x]]
            if len(neighbors) == 0:
                break
            current = neighbors[0]
            thiscycle.append(current)
        cycles.append(thiscycle)
        lengths.append(len(thiscycle))
        if sum(lengths) == n:
            break
            
    return cycles[lengths.index(min(lengths))]

In [14]:
#Creating the model

m= Model()

x = {}
for i in range(n):
    for j in range(i+1):
        x[i,j] = m.addVar(obj= c[(i,j)], vtype=GRB.BINARY, name='X'+str(i+1)+''+str(j+1)) 
        x[j,i] = x[i,j]


NameError: name 'n' is not defined

In [15]:
m.update()

In [16]:
# Add degree-2 constraint, and forbid loops
for i in range(n):
    m.addConstr(quicksum(x[i,j] for j in range(n)) == 2)
    x[i,i].ub = 0

NameError: name 'n' is not defined

In [17]:
m.update()

In [18]:
# Optimize model

m._x = x
m.params.LazyConstraints = 1
m.optimize(subtourelim)

solution = m.getAttr('x', x)
selected = [(i,j) for i in range(n) for j in range(n) if solution[i,j] > 0.5]
assert len(subtour(selected)) == n


Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Optimize a model with 0 rows, 0 columns and 0 nonzeros
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds
Optimal objective  0.000000000e+00


NameError: name 'n' is not defined

In [19]:
#Printing the optimal tour and the optimal value
tour= subtour(selected)
tour= [x+1 for x in tour] #We need to do this since indexing in python starts from zero
print('Optimal tour: %s' % str(tour))
print("\n")
print('Optimal cost: %g' % m.objVal)



NameError: name 'selected' is not defined

# Data visualisation

In [20]:
#Visualisation attempt using an open source TSP code
#However, this method does not work very properly since it is not suitable for large scale problems and not compatible for latitude and longitude values
#https://gist.github.com/payoung/6087046

def plotTSP(paths, points, num_iters=1):
    """
    path: List of lists with the different orders in which the nodes are visited
    points: coordinates for the different nodes
    num_iters: number of paths that are in the path list

    """

    # Unpack the primary TSP path and transform it into a list of ordered
    # coordinates

    x = []
    y = []
    for i in range(len(paths)):
        x.append(points[i][0])
        y.append(points[i][1])

    plt.plot(x, y, 'co')

    # Set a scale for the arrow heads 
    a_scale = float(max(x)) / float(100)

    # Draw the older paths, if provided
    if num_iters > 1:

        for i in range(1, num_iters):

            # Transform the old paths into a list of coordinates
            xi = [];
            yi = [];
            for j in paths[i]:
                xi.append(points[j][0])
                yi.append(points[j][1])

            plt.arrow(xi[-1], yi[-1], (xi[0] - xi[-1]), (yi[0] - yi[-1]),
                      head_width=a_scale, color='r',
                      length_includes_head=True, ls='dashed',
                      width=0.001 / float(num_iters))
            for i in range(0, len(x) - 1):
                plt.arrow(xi[i], yi[i], (xi[i + 1] - xi[i]), (yi[i + 1] - yi[i]),
                          head_width=a_scale, color='r', length_includes_head=True,
                          ls='dashed', width=0.001 / float(num_iters))

    # Draw the primary path for the TSP problem
    plt.arrow(x[-1], y[-1], (x[0] - x[-1]), (y[0] - y[-1]), head_width=a_scale,
              color='g', length_includes_head=True)
    for i in range(0, len(x) - 1):
        plt.arrow(x[i], y[i], (x[i + 1] - x[i]), (y[i + 1] - y[i]), head_width=a_scale,
                  color='g', length_includes_head=True)

    # Set axis too slitghtly larger than the set of x and y
    plt.xlim(0, max(x) * 1.1)
    plt.ylim(0, max(y) * 1.1)
    plt.show()



# Create a randomn list of coordinates, pack them into a list
x_cor = []
y_cor = []
for i in range(n):
    x_cor.append(df["Latitude"][i])
    y_cor.append(df["Longitude"][i])

points = []
for i in range(0, len(x_cor)):
    points.append((x_cor[i], y_cor[i]))

# Run the function
plotTSP(tour, points, 1)


NameError: name 'n' is not defined

# A better method

In [21]:
#We found a better library named folium for our case
#With folium, we can mark nodes on a world map and draw lines between them.

#Creating an empty map
m = folium.Map(location=[20, 0], tiles="OpenStreetMap", zoom_start=2)
 
#We add each nodes marker iteratively
for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['Latitude'], df.iloc[i]['Longitude']]).add_to(m)

#Now we add lines according to the optimal tour
lines= []
for i in range(len(tour)):
    lines.append((df["Latitude"][tour[i]-1],df["Longitude"][tour[i]-1]))
    
#Adding the last line between the last city and the first city for creating a cycle    
lines.append((df["Latitude"][tour[0]-1],df["Longitude"][tour[0]-1]))


folium.PolyLine(lines).add_to(m) #Drawing lines into the map


m.save('TSP_visualisation2.html') #Saving the file into the directory


NameError: name 'df' is not defined